In [ ]:
import os

import nltk
nltk.download('punkt')
import numpy as np
from tqdm.notebook import tqdm

from hse_dialog_tree.utils.files import load_pickle, dump_pickle
from hse_dialog_tree.utils.cpu import get_processor_info
from hse_dialog_tree.utils.batch import Batch
from hse_dialog_tree.text_vectorizers import TfUniversalSentenceEncoder

print(get_processor_info())
!nvidia-smi

In [ ]:
sentence_encoder = TfUniversalSentenceEncoder('models/universal-sentence-encoder-multilingual-large_3')

In [ ]:
for lang in tqdm(os.listdir('data')):
    if not os.path.isdir(f'data/{lang}') or os.path.exists(f'data/{lang}/02_sentence_to_vector.pkl.zip'):
        continue
    print(f'Process {lang}')
    dramas = load_pickle(f'data/{lang}/content_all.pkl.zip')

    dramas_texts = []
    for k, v in dramas.items():
        dramas_texts.append(v)

    only_sentences = []

    for drama in dramas_texts:
        for part in drama:
            only_sentences = only_sentences + [sent for _, sent in part]

    # Also drop too long sentences
    only_sentences = [sent for sent in only_sentences if 0 < len(nltk.word_tokenize(sent)) < 700]
    dump_pickle(only_sentences, f'data/{lang}/01_only_sentences.pkl.zip')

    only_sentences_uniq = list(set(only_sentences))
    vectors = []
    for sentences in tqdm(Batch(only_sentences_uniq, 128)):
        vectors.extend(sentence_encoder(sentences))
    vectors = np.array(vectors)
    sentence_to_vector = {sent: vec for sent, vec in zip(only_sentences_uniq, vectors)}

    dump_pickle(vectors, f'data/{lang}/02_vectors.pkl.zip')
    dump_pickle(sentence_to_vector, f'data/{lang}/02_sentence_to_vector.pkl.zip')


In [ ]:
for lang in tqdm(os.listdir('data')):
    if not os.path.isdir(f'data/{lang}') or os.path.exists(f'data/{lang}/02_sentence_to_vector.pkl.zip'):
        continue
    print(f'Process {lang}')
    dramas = load_pickle(f'data/{lang}/content_all.pkl.zip')

    dramas_texts = []
    for k, v in dramas.items():
        dramas_texts.append(v)

    only_sentences = []

    for drama in dramas_texts:
        for part in drama:
            only_sentences = only_sentences + [sent for _, sent in part]

    # Also drop too long sentences
    only_sentences = [sent for sent in only_sentences if 0 < len(nltk.word_tokenize(sent)) < 700]
    dump_pickle(only_sentences, f'data/{lang}/01_only_sentences.pkl.zip')

    vectors = []
    for sentences in tqdm(Batch(only_sentences, 128)):
        vectors.extend(sentence_encoder(sentences))
    vectors = np.array(vectors)
    sentence_to_vector = {sent: vec for sent, vec in zip(only_sentences, vectors)}

    dump_pickle(vectors, f'data/{lang}/02_vectors_v2.pkl.zip')
    dump_pickle(sentence_to_vector, f'data/{lang}/02_sentence_to_vector_v2.pkl.zip')
